In [1]:
import numpy as np
import pandas as pd
from tensorflow import keras

In [2]:
pred_cols = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C', 'deg_pH10', 'deg_50C']

In [3]:
input_cols = ['sequence', 'structure', 'predicted_loop_type']

In [4]:
train = pd.read_json('train.json', lines=True)
test = pd.read_json('test.json', lines=True)

In [5]:
train = train.query("signal_to_noise >= 1")

In [6]:
train = train.query("SN_filter == 1")

In [7]:
test_private = test.query("seq_length == 130")

In [8]:
test_public = test.query("seq_length == 107")

In [9]:
def preprocess_inputs(df,input_cols):
    """
    Converts inputs into one-hot
    """
    output = []
    for i in range(len(input_cols)):
        tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
        tokenizer.fit_on_texts(np.asarray(df[input_cols[i]]))
        tmp = tokenizer.texts_to_sequences(np.asarray(df[input_cols[i]]))
        output.append(np.asarray(keras.utils.to_categorical(tmp)[:,:,1:]))
    return output

In [10]:
inputs = preprocess_inputs(train,input_cols)

In [11]:
len(inputs[0])

1587

In [12]:
def merge_inputs(inputs, length):
    """
    Merges the one-hot inputs by columns
    Also snips seq length's till desired amount
    """
    size = len(inputs[0])
    output = []
    for i in range(size):
        output.append(np.concatenate((inputs[0][i][0:length], inputs[1][i][0:length], inputs[2][i][0:length]), axis = 1))
    return np.asarray(output)

In [13]:
inputs_simple = merge_inputs(inputs, 68)

In [14]:
inputs_simple.shape

(1587, 68, 14)

In [15]:
test1 = np.zeros((2,5,5))

In [16]:
test1[1,1,0:3]

array([0., 0., 0.])

In [17]:
def preprocess_results(df, results):
    """
    Makes sure that the results are in the appropriate format:
        [layers,lines,columns] in an np array
    """
    tmp = np.asarray(df[results])
    size = len(tmp[0])
    output = np.zeros((len(tmp),len(tmp[0][0]),len(results)))
    for i in range(len(results)):
        for j in range(size):
            tmp[i,j] = np.asarray(tmp[i,j])
    for i in range(len(tmp)):
        output[i] = np.vstack((tmp[i,0], tmp[i,1], tmp[i,2], 
                                tmp[i,3], tmp[i,4]))[:,:].T
    return output

In [18]:
expected_results = preprocess_results(train, pred_cols)

In [19]:
expected_results.shape

(1587, 68, 5)

In [20]:
expected_results[0,0]

array([0.3297, 0.7556, 0.3581, 2.3375, 0.6382])

In [34]:
model_simple = keras.Sequential()

model_simple.add(keras.layers.Conv1D(filters=10, kernel_size=3, activation=('relu'), 
                                     input_shape=(None,14), padding='same'))
model_simple.add(keras.layers.GlobalMaxPooling1D())
model_simple.add(keras.layers.Dense(20, activation=('relu')))
model_simple.add(keras.layers.Dense(16, activation='relu'))
model_simple.add(keras.layers.Dropout(rate=0.4))
model_simple.add(keras.layers.Dense(10, activation='relu'))


#Est-ce que le out put c'est 3 valuers distinctes, ou 1 valeurs mais qui existe dans 3 channels?
model_simple.add(keras.layers.Dense(68, activation='linear'))

# mean_squared_error car on a affair a une regression
model_simple.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])
model_simple.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_3 (Conv1D)            (None, None, 10)          430       
_________________________________________________________________
global_max_pooling1d_3 (Glob (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 20)                220       
_________________________________________________________________
dense_13 (Dense)             (None, 16)                336       
_________________________________________________________________
dropout_3 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 10)                170       
_________________________________________________________________
dense_15 (Dense)             (None, 68)                748       
Total para

In [35]:
model_simple.fit(inputs_simple[:], expected_results[:,:,0], batch_size=64, 
                 epochs=100, verbose=1, validation_split=0.2) # validation loss keeps going down?

Train on 1269 samples, validate on 318 samples
Epoch 1/100
1269/1269 [==============================] - 2s 1ms/step - loss: 0.3404 - mean_squared_error: 0.3404 - val_loss: 0.3214 - val_mean_squared_error: 0.3214
Epoch 2/100
1269/1269 [==============================] - 0s 347us/step - loss: 0.3030 - mean_squared_error: 0.3030 - val_loss: 0.2796 - val_mean_squared_error: 0.2796
Epoch 3/100
1269/1269 [==============================] - 0s 331us/step - loss: 0.2669 - mean_squared_error: 0.2669 - val_loss: 0.2390 - val_mean_squared_error: 0.2390
Epoch 4/100
1269/1269 [==============================] - 0s 260us/step - loss: 0.2340 - mean_squared_error: 0.2340 - val_loss: 0.2079 - val_mean_squared_error: 0.2079
Epoch 5/100
1269/1269 [==============================] - 0s 244us/step - loss: 0.2101 - mean_squared_error: 0.2101 - val_loss: 0.1887 - val_mean_squared_error: 0.1887
Epoch 6/100
1269/1269 [==============================] - 0s 268us/step - loss: 0.1952 - mean_squared_error: 0.1952 - val

1269/1269 [==============================] - 0s 347us/step - loss: 0.1573 - mean_squared_error: 0.1573 - val_loss: 0.1621 - val_mean_squared_error: 0.1621
Epoch 50/100
1269/1269 [==============================] - 0s 299us/step - loss: 0.1570 - mean_squared_error: 0.1570 - val_loss: 0.1615 - val_mean_squared_error: 0.1615
Epoch 51/100
1269/1269 [==============================] - 0s 276us/step - loss: 0.1568 - mean_squared_error: 0.1568 - val_loss: 0.1612 - val_mean_squared_error: 0.1612
Epoch 52/100
1269/1269 [==============================] - 0s 236us/step - loss: 0.1565 - mean_squared_error: 0.1565 - val_loss: 0.1616 - val_mean_squared_error: 0.1616
Epoch 53/100
1269/1269 [==============================] - 0s 236us/step - loss: 0.1564 - mean_squared_error: 0.1564 - val_loss: 0.1609 - val_mean_squared_error: 0.1609
Epoch 54/100
1269/1269 [==============================] - 0s 236us/step - loss: 0.1560 - mean_squared_error: 0.1560 - val_loss: 0.1610 - val_mean_squared_error: 0.1610
Epoch

Epoch 98/100
1269/1269 [==============================] - 0s 284us/step - loss: 0.1511 - mean_squared_error: 0.1511 - val_loss: 0.1562 - val_mean_squared_error: 0.1562
Epoch 99/100
1269/1269 [==============================] - 0s 236us/step - loss: 0.1510 - mean_squared_error: 0.1510 - val_loss: 0.1562 - val_mean_squared_error: 0.1562
Epoch 100/100
1269/1269 [==============================] - 0s 244us/step - loss: 0.1508 - mean_squared_error: 0.1508 - val_loss: 0.1563 - val_mean_squared_error: 0.1563


In [38]:
model_simple.predict(inputs_simple)

array([[0.49903202, 1.3673505 , 1.0804473 , ..., 0.3443737 , 0.30548564,
        0.2879229 ],
       [0.46239996, 1.2894375 , 1.1232172 , ..., 0.4509542 , 0.3730848 ,
        0.29419088],
       [0.46583292, 1.242483  , 0.95405495, ..., 0.29894933, 0.26896116,
        0.27414298],
       ...,
       [0.55453503, 1.6591792 , 1.3614894 , ..., 0.47080028, 0.415768  ,
        0.3062266 ],
       [0.3492043 , 0.9457277 , 0.8343984 , ..., 0.3986888 , 0.2997829 ,
        0.29006147],
       [0.5204314 , 1.3829736 , 0.9764974 , ..., 0.23767503, 0.24073514,
        0.26185977]], dtype=float32)

In [25]:
inputs_simple_test_public = preprocess_inputs(test_public,input_cols)
inputs_simple_test_public = merge_inputs(inputs_simple_test_public, len(inputs_simple_test_public[0][0]))

In [26]:
inputs_simple_test_public.shape

(629, 107, 14)

In [39]:
model_simple.predict(inputs_simple_test_public)

array([[0.43536642, 1.1760983 , 1.0193015 , ..., 0.39466888, 0.3210688 ,
        0.28773066],
       [0.4054707 , 1.1344357 , 1.0132607 , ..., 0.4402719 , 0.35577345,
        0.29849964],
       [0.3997292 , 1.088759  , 0.99809575, ..., 0.4348591 , 0.33852947,
        0.2902928 ],
       ...,
       [0.42186713, 1.1826756 , 1.0807523 , ..., 0.47631603, 0.37483597,
        0.29756004],
       [0.4412241 , 1.2346762 , 1.093946  , ..., 0.44991648, 0.3658199 ,
        0.2966246 ],
       [0.42913425, 1.1476092 , 0.9673294 , ..., 0.36187583, 0.29775298,
        0.2868853 ]], dtype=float32)

In [24]:
expected_results[:,:,0]

array([[ 0.3297,  1.5693,  1.1227, ...,  0.2937,  0.2362,  0.5731],
       [ 0.4482,  1.4822,  1.1819, ...,  0.6449,  0.04  ,  0.5446],
       [ 0.7642,  1.6641,  1.0622, ...,  0.1107,  0.2261,  0.3238],
       ...,
       [ 0.6957,  1.251 ,  1.3236, ..., -0.0043,  0.0521,  0.0874],
       [ 0.2891,  0.4496,  0.7165, ...,  0.8738,  0.2816,  0.554 ],
       [ 1.0102,  1.7928,  1.9228, ...,  0.0381, -0.0066,  0.0706]])

In [ ]:
expected_results[1]

In [40]:
seq_input = keras.layers.Input(shape=(None,4), name="seq")      #Input shape a revoir
pair_input = keras.layers.Input(shape=(None,3), name="pair")  
loop_input = keras.layers.Input(shape=(None,7), name="loop")  

seq_features = keras.layers.Conv1D(filters=10, kernel_size=3, activation=('relu'), 
                                     padding='same')(seq_input)
pair_features = keras.layers.Conv1D(filters=10, kernel_size=3, activation=('relu'), 
                                     padding='same')(pair_input)
loop_features = keras.layers.Conv1D(filters=10, kernel_size=3, activation=('relu'), 
                                     padding='same')(loop_input)

seq_features = keras.layers.GlobalMaxPooling1D()(seq_features)
pair_features = keras.layers.GlobalMaxPooling1D()(pair_features)
loop_features = keras.layers.GlobalMaxPooling1D()(loop_features)

seq_features = keras.layers.Dense(20)(seq_features)
pair_features = keras.layers.Dense(20)(pair_features)
loop_features = keras.layers.Dense(20)(loop_features)


# Merge les features
x = keras.layers.concatenate([seq_features, pair_features, loop_features])

flat = keras.layers.Flatten()(x)

first_pred = keras.layers.Dense(68, name="reactivity")(flat) #regression pour "reactivity"
second_pred = keras.layers.Dense(68, name="deg_Mg_pH10")(flat)  #regression pour "ph"
third_pred = keras.layers.Dense(68, name="deg_Mg_50C")(flat)
fourth_pred = keras.layers.Dense(68, name="deg_pH10")(flat)
fifth_pred = keras.layers.Dense(68, name="deg_50C")(flat)


model = keras.Model(
    inputs=[seq_input, pair_input, loop_input],
    outputs=[first_pred, second_pred, third_pred, fourth_pred, fifth_pred],
)
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
seq (InputLayer)                (None, None, 4)      0                                            
__________________________________________________________________________________________________
pair (InputLayer)               (None, None, 3)      0                                            
__________________________________________________________________________________________________
loop (InputLayer)               (None, None, 7)      0                                            
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, None, 10)     130         seq[0][0]                        
__________________________________________________________________________________________________
conv1d_5 (

In [41]:
model.fit([inputs_simple[:,:,0:4], inputs_simple[:,:,4:7], inputs_simple[:,:,7:14]],
          [expected_results[:,:,0], expected_results[:,:,1], expected_results[:,:,2],
          expected_results[:,:,3], expected_results[:,:,4]], batch_size = 25, 
          epochs = 30, verbose = 1, validation_split = 0.2)

Train on 1269 samples, validate on 318 samples
Epoch 1/30
1269/1269 [==============================] - 4s 3ms/step - loss: 1.5829 - reactivity_loss: 0.3114 - deg_Mg_pH10_loss: 0.3016 - deg_Mg_50C_loss: 0.3010 - deg_pH10_loss: 0.3943 - deg_50C_loss: 0.2745 - reactivity_mean_squared_error: 0.3114 - deg_Mg_pH10_mean_squared_error: 0.3016 - deg_Mg_50C_mean_squared_error: 0.3010 - deg_pH10_mean_squared_error: 0.3943 - deg_50C_mean_squared_error: 0.2745 - val_loss: 0.9488 - val_reactivity_loss: 0.1828 - val_deg_Mg_pH10_loss: 0.2169 - val_deg_Mg_50C_loss: 0.1823 - val_deg_pH10_loss: 0.2213 - val_deg_50C_loss: 0.1457 - val_reactivity_mean_squared_error: 0.1828 - val_deg_Mg_pH10_mean_squared_error: 0.2169 - val_deg_Mg_50C_mean_squared_error: 0.1823 - val_deg_pH10_mean_squared_error: 0.2213 - val_deg_50C_mean_squared_error: 0.1457
Epoch 2/30
1269/1269 [==============================] - 1s 630us/step - loss: 0.8685 - reactivity_loss: 0.1686 - deg_Mg_pH10_loss: 0.2127 - deg_Mg_50C_loss: 0.1769 - d

1269/1269 [==============================] - 1s 615us/step - loss: 0.8107 - reactivity_loss: 0.1606 - deg_Mg_pH10_loss: 0.2073 - deg_Mg_50C_loss: 0.1689 - deg_pH10_loss: 0.1510 - deg_50C_loss: 0.1229 - reactivity_mean_squared_error: 0.1606 - deg_Mg_pH10_mean_squared_error: 0.2073 - deg_Mg_50C_mean_squared_error: 0.1689 - deg_pH10_mean_squared_error: 0.1510 - deg_50C_mean_squared_error: 0.1229 - val_loss: 0.8145 - val_reactivity_loss: 0.1634 - val_deg_Mg_pH10_loss: 0.2080 - val_deg_Mg_50C_loss: 0.1676 - val_deg_pH10_loss: 0.1508 - val_deg_50C_loss: 0.1246 - val_reactivity_mean_squared_error: 0.1634 - val_deg_Mg_pH10_mean_squared_error: 0.2080 - val_deg_Mg_50C_mean_squared_error: 0.1676 - val_deg_pH10_mean_squared_error: 0.1508 - val_deg_50C_mean_squared_error: 0.1246
Epoch 12/30
1269/1269 [==============================] - 1s 599us/step - loss: 0.8086 - reactivity_loss: 0.1603 - deg_Mg_pH10_loss: 0.2069 - deg_Mg_50C_loss: 0.1684 - deg_pH10_loss: 0.1506 - deg_50C_loss: 0.1224 - reactivit

1269/1269 [==============================] - 1s 693us/step - loss: 0.7885 - reactivity_loss: 0.1561 - deg_Mg_pH10_loss: 0.2031 - deg_Mg_50C_loss: 0.1639 - deg_pH10_loss: 0.1472 - deg_50C_loss: 0.1182 - reactivity_mean_squared_error: 0.1561 - deg_Mg_pH10_mean_squared_error: 0.2031 - deg_Mg_50C_mean_squared_error: 0.1639 - deg_pH10_mean_squared_error: 0.1472 - deg_50C_mean_squared_error: 0.1182 - val_loss: 0.7986 - val_reactivity_loss: 0.1599 - val_deg_Mg_pH10_loss: 0.2046 - val_deg_Mg_50C_loss: 0.1643 - val_deg_pH10_loss: 0.1480 - val_deg_50C_loss: 0.1218 - val_reactivity_mean_squared_error: 0.1599 - val_deg_Mg_pH10_mean_squared_error: 0.2046 - val_deg_Mg_50C_mean_squared_error: 0.1643 - val_deg_pH10_mean_squared_error: 0.1480 - val_deg_50C_mean_squared_error: 0.1218
Epoch 22/30
1269/1269 [==============================] - 1s 930us/step - loss: 0.7861 - reactivity_loss: 0.1555 - deg_Mg_pH10_loss: 0.2027 - deg_Mg_50C_loss: 0.1634 - deg_pH10_loss: 0.1468 - deg_50C_loss: 0.1177 - reactivit

In [ ]:
inputs_simple[0,0,0:4]


**Revoir donnees a predire**